In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import cycle
import etc

import sys
sys.path.append("../src")

import Stromingen.Munsflow_H2O_1995.src.dutch_soils as ds

ModuleNotFoundError: No module named 'dutch_soils'

In [ ]:
dirs = etc.Dirs('~/GRWMODELS/python/tools/Stromingen/Munsflow_H2O_1995')

In [ ]:
wbook = os.path.join(dirs.data, 'VG_soilprops.xlsx')
ds.Soil.load_soils(wbook) # load once
ds.Soil.pretty_data()
sand_b = ds.Soil("O01")
sand_o = ds.Soil("B01")

In [ ]:
# %% psi(theta)
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title=r"$\psi(\theta)$", xlabel=r'$\theta$', ylabel=r'$\psi$ [cm]')
ax.set_yscale('log')
ax.grid(True)

clrs = cycle("brgkmcy")
for code in ds.Soil.data.index:
    clr = next(clrs)
    soil = ds.Soil(code)
    if soil.props['Hoofdsoort'] != 'Zand':
        continue
    theta = np.linspace(soil.props['theta_r'], soil.props['theta_s'])[1:-1]
    label=f"soil[{code}]:  {soil.props['Omschrijving']}" if code.endswith('01') else ""
    ax.plot(theta, soil.psi_fr_theta(theta), color=clr, label=label)
    
    pF_fc, pF_wp = 2.5, 4.2
    lbl1 = f"fc[pF={pF_fc}]" if code.endswith('01') else ""
    lbl2 = f"fc[pF={pF_wp}]" if code.endswith('01') else ""
    ax.plot(soil.theta_fc(pF=pF_fc), 10 ** pF_fc, 'o', color=clr, label=lbl1)
    ax.plot(soil.theta_wp(pF=pF_wp), 10 ** pF_wp, 's', color=clr, label=lbl2)
ax.legend(title="Legenda voor alleen B01 en O01")

In [ ]:
# %% psi(theta)
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title="retention curves for sand_b", xlabel='theta', ylabel='pF [log(cm)]')
ax.set_yscale('log')
ax.grid(which='both')

theta = np.linspace(sand_b.props['theta_r'], sand_b.props['theta_s'], 100)
psi = np.logspace(0, 4.2, 100)

ax.plot(sand_b.theta_fr_psi(psi), psi, label="theta fr psi")
ax.plot(theta, sand_b.psi_fr_theta(theta), '.', label="psi fr theta")
ax.legend()

In [ ]:
# %% K(theta)
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title=r"$K$ versus $\theta$", xlabel=r'$\theta$', ylabel=r'$K(\theta)$')
ax.set_yscale('log')
ax.grid(True)

for code in ds.Soil.data.index:
    soil = ds.Soil(code)
    if soil.props['Hoofdsoort'].lower() != 'zand':
        continue
    theta = np.linspace(soil.props['theta_r'], soil.props['theta_s'])[1:-1]
    ax.plot(theta, soil.K(theta), label=f"soil[{code}]: {soil.props['Omschrijving']}")
ax.legend()

In [ ]:
# %% K(theta)
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title=r"$dK/d\theta$", xlabel=r'$\theta$', ylabel=r'$dK/d\theta$')
ax.set_yscale('log')
ax.set_yscale('linear')
ax.grid(True)

theta = np.linspace(sand_b.props['theta_r'], sand_b.props['theta_s'], 100)[1:-1]

d = 1e-3
dkdtheta_num = (sand_b.K(theta + d) - sand_b.K(theta - d)) / (2 * d)

ax.plot(theta, sand_b.dK_dtheta(theta), label="analytical")
ax.plot(theta, dkdtheta_num, '.', label="numeric")
ax.legend()

In [ ]:
# %% Show field capacities and wilting points
for BO in ['Boverngronden', 'Ondergronden']:
    title = f'Veldcapaciteit en Verwelkingspunt van {BO} van de Staringreeks'
    xlabel = 'soil code'
    ylabel = r'$\theta$'
    ax = etc.newfig(title, xlabel, ylabel)

    fcs, wps, codes = [], [], []
    for code in [c for c in ds.Soil.data.index if c.startswith(BO[0])]:
        soil = ds.Soil(code)
        codes.append(code)
        fcs.append(soil.theta_fc())
        wps.append(soil.theta_wp())
        
        i = int(code[1:])
        ax.text(int(code[1:]) - 1, 0.15,
                soil.props['Omschrijving'], ha='center', rotation=90, fontsize=15, zorder=5,
                bbox=dict(facecolor='white',
                        alpha=1,
                        edgecolor='none'))

    ax.plot(codes, np.array(fcs), 'b', label='Veldcapaciteit')
    ax.plot(codes, np.array(wps), 'r', label='Verwelkingspunt')
    ax.legend(fontsize=15, loc='lower right')

    #ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.xticks(rotation=90, ha="center")
    plt.tight_layout()

In [ ]:
# %% K(theta) bij veldcapaciteit (pF=2.5)
for BO in ['Boverngronden', 'Ondergronden']:
    title = f'K(theta) bij Veldcapaciteit van {BO} van de Staringreeks'
    xlabel = 'soil code'
    ylabel = 'K [cm/d]$'
    ax = etc.newfig(title, xlabel, ylabel)

    fcs, wps, kvalues, codes = [], [], [], []
    for code in [c for c in ds.Soil.data.index if c.startswith(BO[0])]:
        soil = ds.Soil(code)
        codes.append(code)
        K = soil.K(soil.theta_fc(pF=2.5))
        kvalues.append(K)
        
        i = int(code[1:])
        ax.text(int(code[1:]) - 1, 0.01,
                soil.props['Omschrijving'], ha='center', rotation=90, fontsize=15, zorder=5,
                bbox=dict(facecolor='white',
                        alpha=1,
                        edgecolor='none'))

    ax.plot(codes, np.array(kvalues), 'b', label='k bij velccapaciteit')    
    ax.legend(fontsize=15, loc='lower right')

    #ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.xticks(rotation=90, ha="center")
    plt.tight_layout()

In [ ]:
# %% Show the throttle function to throttle ET depending on theta
soil = sand_o

title = f"Throttle functions(theta) for ET  for soil {soil.props['Omschrijving']}"
ax = etc.newfig(title, r"$\theta$", r"throttle $\alpha(\theta)$")

theta = np.linspace(soil.props['theta_r'], soil.props['theta_s'], 200)

# ax.plot(theta, soil.alpha_wet(theta), label=r'alpha_wett')
ax.plot(theta, soil.feddes_alpha(theta), label='feddes_alpha')
ax.plot(theta, soil.smooth_alpha(theta), label='smooth_alpha')
beta = 1.0
for w in [0.1, 0.25, 0.5, 0.75, 1.0]:
    ax.plot(theta, soil.alpha_dry_blend(theta, beta=beta, w=w), '-.',
        label=f'alpha_dry_blend, beta={beta}, w={w}')
for beta in [1.0]:
        ax.plot(theta, soil.mualem_alpha(theta, beta=beta), '.-', label=f'mualem_alpha, beta={beta}')
ax.legend()
plt.show()